In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import subprocess
import re

# Esegui il comando 'pip freeze' e ottieni l'output
result = subprocess.run(['pip', 'freeze'], stdout=subprocess.PIPE)
packages = result.stdout.decode().splitlines()

# Filtrare le linee che contengono "file://"
filtered_packages = [package for package in packages if not re.search(r'file://', package)]

# Scrivere i pacchetti filtrati nel file 'requirements.txt'
with open('requirements.txt', 'w') as file:
    for package in filtered_packages:
        file.write(package + '\n')

print("Il file requirements.txt è stato creato senza stringhe 'file:///home/...'.")


In [ ]:
!pip install langchain chromadb langchain_community pypdf

In [ ]:
!pip install ragas

In [ ]:
pip install groq

In [ ]:
pip install langchain-groq==0.1.6

In [ ]:
pip install openai

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA, LLMChain
from langchain_groq import ChatGroq

In [ ]:
import os
#imposto la chiavi per i servizi che utilizzerò
os.environ["GROQ_API_KEY"] = "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"
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ["AZURE_OPENAI_API_KEY"] = "21a4eae201494ec581d92fbeb45891b3" #Mbesi Open 9d7cdcf0ed72472dbf75f7e02a25d506 - # Mbesi est 21a4eae201494ec581d92fbeb45891b3

# IMPOSTO AZURE OPENAI

In [ ]:
azure_configs = {
    "base_url": "https://mbesiest.openai.azure.com/", # Mbesi open https://mbesiopen.openai.azure.com/ - Mbesi est https://mbesiest.openai.azure.com/
    "model_deployment": "gpt-4o-mini",
    "model_name": "gpt-4o-mini",
    "embedding_deployment": "text-embedding-ada-002",
    "embedding_name": "text-embedding-ada-002",  # most likely  text-embedding-ada-002
}

In [ ]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model_GPT3 = AzureChatOpenAI(
    openai_api_version="2024-06-01",
    azure_endpoint=azure_configs["base_url"],
    azure_deployment="gpt-35-turbo",
    model="gpt-35-turbo",
    validate_base_url=False,
)
azure_model_GPT4OMINI = AzureChatOpenAI(
    openai_api_version="2024-06-01",
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["model_deployment"],
    model=azure_configs["model_name"],
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_version="2024-02-01",
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["embedding_deployment"],
    model=azure_configs["embedding_name"],
)

# INPUT E CHUNK DEL FILE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

NUOVO DATASET COMPLETO

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='documenti/DomandeRisposteNew.csv', csv_args={
    'delimiter': ';',
    'quotechar': '"',
    'fieldnames': ['DOMANDE', 'RISPOSTE']
})

docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=51) #300 - 50/ 250 - 50/ 256 - 25/ 300 - 30

chunks = text_splitter.split_documents(docs)




In [ ]:
len(chunks)

In [ ]:
# RAGAS expects a file_name dict as key
for document in chunks:
    document.metadata['file_name'] = document.metadata['source']

# CON LLM OPEN

## IMPORTO LIBRERIE - CREO VECTORSTORE, PROMPT E RETRIEVER

In [ ]:
import torch
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")


In [ ]:
"""
SE MI TROVO SU COLAB INSTALLO IL TERMINALE PER OLLAMA
#!pip install colab-xterm
%load_ext colabxterm
%xterm
"""

In [ ]:
# scrivo le seguenti linee in entrambi i casi:
#1- curl -faSL https://ollama.com/install.sh | sh
#2- ollama serve & ollama pull gemma2
#3- ollama pull nomic-embed-text
#4- ollama pull llama3

In [ ]:
# importo le librerie che utilizzerò
import pandas as pd

from langchain.document_loaders import DirectoryLoader # se carico direttamente una cartella di documenti
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings # se utilizzo openai
from langchain_community.embeddings import OllamaEmbeddings


from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
#embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [ ]:
from tqdm import tqdm
import time

# Simuliamo il tempo di esecuzione per la dimostrazione
start_time = time.time()

# Crea un oggetto tqdm per monitorare l'avanzamento
with tqdm(total=100) as pbar:
    # Simulazione dell'elaborazione (ad esempio per un loop sui chunks)
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db/med256_db")
    
    # Aggiornamento della barra di caricamento
    pbar.update(100)

# Calcolo del tempo di esecuzione
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo di esecuzione: {execution_time:.2f} secondi")


In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db/med256_db")
#, persist_directory="./chroma_db/txt_db"

In [ ]:
# In caso ho l'ho già creato, carico database vettoriale
vectorstore = Chroma(persist_directory="./chroma_db/med_db", embedding_function = embeddings) #med256 per il vectorstore diviso in 256 chunk


In [ ]:
#dimostro che il caricamento è avvenuto correttamente
docum = vectorstore.similarity_search("come curo l'ipertensione?")
print(docum[0].page_content)

In [ ]:
vectorstore

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 3}# ritorna i k risultati simili quando eseguo una ricerca sul documento
)

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.1")

In [ ]:
#PROMPT MIGLIORATO CON GIANLUCA
from langchain_core.prompts import PromptTemplate

template = """Sei un assistente medico IA. Fornisci risposte accurate e dettagliate in italiano, usando un linguaggio chiaro ma appropriato. Rispetta sempre la privacy del paziente.
Se non sei sicuro della risposta, dichiara "Non sicuro della risposta".


Domanda: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
  )

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Sei un assistente medico IA specializzato in ipertensione. Fornisci risposte dettagliate e basate su evidenze scientifiche, mantenendo un linguaggio chiaro e accessibile. Rispetta sempre la privacy del paziente e, se non sei sicuro della risposta, dichiara "Non sicuro della risposta". 
Includi le raccomandazioni attuali e spiega i concetti medici in modo comprensibile.

Domanda: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)


## IMPOSTO TUTTE LE TIPOLOGIE DI RETRIVER

### BASE RETRIEVER

Retriever base è utilizzato per eseguire una ricerca di somiglianza, trovando i documenti più simili alla query basandosi sui loro vettori

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(rag_chain.invoke("Quali strumenti tecnologici mostrano dati storici e attuali dell'ipertensione?"))

In [ ]:
retriever.invoke("Quali strumenti tecnologici mostrano dati storici e attuali dell'ipertensione? ")

Il MultiQueryRetriever automatizza l'ottimizzazione dei prompt utilizzando un LLM per generare più query da diverse prospettive relative a una specifica richiesta dell'utente. Per ciascuna query, il sistema recupera un insieme di documenti rilevanti e poi unisce in modo unico i risultati ottenuti da tutte le query, al fine di ottenere un insieme più ampio di documenti potenzialmente utili. Generando diverse prospettive sulla stessa domanda, il MultiQueryRetriever può superare alcune limitazioni del reperimento basato sulla distanza, fornendo così un insieme di risultati più ricco.

In [ ]:
from langchain.retrievers import MultiQueryRetriever
final_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

multirag_chain = (
    {"context": final_retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(multirag_chain.invoke("Come si cura l'ipertensione?"))

In [ ]:
generator_llm = Ollama(model="mistral-nemo")
critic_llm = Ollama(model="qwen2")

ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
    #model="mxbai-embed-large"
)

print('Generator LLM: '+ generator_llm.invoke('Salutami in italiano'))
print('Critic LLM: '+ critic_llm.invoke('Salutami in italiano e dimmi chi sei'))

r2 = ollama_emb.embed_query(
    "dimmi la seconda lettera dell'alfabeto greco"
)

print(f'Dimensione Embedding: {len(r2)}')

## IMPORTO METRICHE RAGAS

In [ ]:
from ragas import evaluate
from ragas.metrics.critique import harmfulness
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_similarity,
   #context_relevancy,
    context_recall,
    context_precision,
    answer_correctness
)


## VALUTAZIONE CON METRICHE RAGAS -BASE RETRIEVER

In [ ]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

In [ ]:
import pandas as pd

# Carica il CSV in un nuovo DataFrame
loaded_df = pd.read_csv('test_dataChatGPT4O_FINALE.csv') #test_dataGPT4OMINI _BUONO / test_dataChatGPT4O_BUONO (sembra ottenere risultati migliori con gpt4O)

# Estrai le colonne 'question' e 'ground_truth'
questions = loaded_df["question"].to_list()
ground_truth = loaded_df["ground_truth"].to_list()


In [ ]:
#PER CAMBIARE LLM DI VALUTAZIONE TUTTI I RETRIEVER FARANNO RIFERIMENTO
llm = Ollama(model="echelonify/med-qwen2") #echelonify/med-qwen2

In [ ]:
from datasets import Dataset
from tqdm import tqdm

# Assicurati che questions e ground_truth siano definiti
# questions = testset.to_pandas()["question"].to_list()
# ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

# Utilizzo di tqdm per creare una barra di avanzamento
for query in tqdm(questions, desc="Processing queries"):
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append([""])  # Contesto vuoto per ogni elemento

dataset = Dataset.from_dict(data)

In [ ]:
dataset.to_pandas()
datacompleto= dataset.to_pandas()
datacompleto.to_csv('DataCompletoQWENMED5.csv', index=False)
#datacompleto

In [ ]:
#SE LO IMPORTO DAI DATAFRAME PRECEDENTI

import pandas as pd
from datasets import Dataset

# Leggi il file CSV in un DataFrame di Pandas
dataframe = pd.read_csv('DataCompletoQWENMED.csv')

# Converti il DataFrame in un dataset Hugging Face
datasetImportato = Dataset.from_pandas(dataframe)

# Mostra il dataset
print(datasetImportato)


In [ ]:
#PER TESTARE SINGOLA METRICA
score = evaluate(dataset,metrics=[answer_relevancy, faithfulness, answer_correctness], llm=azure_model_GPT4OMINI, embeddings=azure_embeddings)
score.to_pandas()

In [ ]:
score

In [ ]:
DatasetScore= score.to_pandas()
DatasetScore.to_csv('SCOREQWENMED.csv', index=False)

In [ ]:
#score med qwen
score

In [ ]:
%%time
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from ragas import evaluate

#langchain_llm =  # any langchain LLM instance
#langchain_embeddings = # any langchain Embeddings instance

resultBase = evaluate(
        dataset = dataset,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness,
              context_recall,
              context_precision,
              harmfulness
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )


In [ ]:
resultBase.to_pandas()

In [ ]:
from datasets import Dataset
from tqdm import tqdm

# Assicurati che questions e ground_truth siano definiti
# questions = testset.to_pandas()["question"].to_list()
# ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

# Utilizzo di tqdm per creare una barra di avanzamento
for query in tqdm(questions, desc="Processing queries"):
    data["question"].append(query)
    data["answer"].append(multirag_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in retriever.get_relevant_documents(query)])

response_dataset_advanced_retrieval = Dataset.from_dict(data)

In [ ]:
response_dataset_advanced_retrieval.to_pandas()
responseMultiRetriver= response_dataset_advanced_retrieval.to_pandas()
responseMultiRetriver.to_csv('responseMultiRetriver.csv', index=False)
responseMultiRetriver

In [ ]:
#PER TESTARE SINGOLA METRICA
score2 = evaluate(response_dataset_advanced_retrieval,metrics=[answer_relevancy, faithfulness], llm=azure_model_GPT4OMINI, embeddings=azure_embeddings)
score2.to_pandas()

In [ ]:
score2

In [ ]:
score2

In [ ]:
#VALUTO CON CHAT IL MULTIQUERYRETRIVER
resultMulti = evaluate(
        dataset = response_dataset_advanced_retrieval,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness,
              context_recall,
              context_precision,
              harmfulness
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )

In [ ]:
resultMulti.to_pandas()

In [ ]:
from datasets import Dataset
from tqdm import tqdm

# Assicurati che questions e ground_truth siano definiti
# questions = testset.to_pandas()["question"].to_list()
# ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

# Utilizzo di tqdm per creare una barra di avanzamento
for query in tqdm(questions, desc="Processing queries"):
    data["question"].append(query)
    data["answer"].append(ensemble_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in retriever.get_relevant_documents(query)])

response_dataset_ensemble_retrieval = Dataset.from_dict(data)

In [ ]:
#PER TESTARE SINGOLA METRICA
score3 = evaluate(response_dataset_ensemble_retrieval,metrics=[answer_relevancy, faithfulness], llm=azure_model_GPT4OMINI, embeddings=azure_embeddings)
score3.to_pandas()

In [ ]:
score

In [ ]:
score2

In [ ]:
score3

In [ ]:
#VALUTO CON CHAT IL ENSEMBLERETRIVER
resultEnsemble = evaluate(
        dataset = response_dataset_ensemble_retrieval,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness,
              context_recall,
              context_precision,
              harmfulness
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )

In [ ]:
resultEnsemble.to_pandas()

In [ ]:
resultBase

In [ ]:
resultMulti

In [ ]:
resultEnsemble

In [ ]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

In [ ]:
import pandas as pd

# Carica il CSV in un nuovo DataFrame
loaded_df = pd.read_csv('test_dataLLAMA3.csv')

# Estrai le colonne 'question' e 'ground_truth'
questions = loaded_df["question"].to_list()
ground_truth = loaded_df["ground_truth"].to_list()

In [ ]:
#PER CAMBIARE LLM DI VALUTAZIONE TUTTI I RETRIEVER FARANNO RIFERIMENTO
llm = Ollama(model="llama3.1:70b")

In [ ]:
from datasets import Dataset
from tqdm import tqdm

# Assicurati che questions e ground_truth siano definiti
# questions = testset.to_pandas()["question"].to_list()
# ground_truth = testset.to_pandas()["ground_truth"].to_list()

# Inizializzazione dei tre dizionari
dataset = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}
response_dataset_advanced_retrieval = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}
response_dataset_ensemble_retrieval = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

# Utilizzo di tqdm per creare una barra di avanzamento
for query in tqdm(questions, desc="Processing queries"):
    # Recupero dei documenti rilevanti (assumendo che sia lo stesso per tutte le catene)
    relevant_docs = retriever.get_relevant_documents(query)
    contexts = [doc.page_content for doc in relevant_docs]

    # Popolare il dataset originale
    dataset["question"].append(query)
    dataset["answer"].append(rag_chain.invoke(query))
    dataset["contexts"].append(contexts)

    # Popolare il dataset per advanced retrieval
    response_dataset_advanced_retrieval["question"].append(query)
    response_dataset_advanced_retrieval["answer"].append(multirag_chain.invoke(query))
    response_dataset_advanced_retrieval["contexts"].append(contexts)

    # Popolare il dataset per ensemble retrieval
    response_dataset_ensemble_retrieval["question"].append(query)
    response_dataset_ensemble_retrieval["answer"].append(ensemble_chain.invoke(query))
    response_dataset_ensemble_retrieval["contexts"].append(contexts)

# Convertire i dizionari in Dataset
dataset = Dataset.from_dict(dataset)
response_dataset_advanced_retrieval = Dataset.from_dict(response_dataset_advanced_retrieval)
response_dataset_ensemble_retrieval = Dataset.from_dict(response_dataset_ensemble_retrieval)

# Convertire in DataFrame se necessario
dataset_df = dataset.to_pandas()
advanced_retrieval_df = response_dataset_advanced_retrieval.to_pandas()
ensemble_retrieval_df = response_dataset_ensemble_retrieval.to_pandas()

In [ ]:
%%time
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from ragas import evaluate

#langchain_llm =  # any langchain LLM instance
#langchain_embeddings = # any langchain Embeddings instance

resultBase = evaluate(
        dataset = dataset,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness#,
              #context_recall,
              #context_precision
              #context_relevancy
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )

resultMulti = evaluate(
        dataset = response_dataset_advanced_retrieval,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness#,
              #context_recall,
              #context_precision
              #context_relevancy
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )

resultEnsemble = evaluate(
        dataset = response_dataset_ensemble_retrieval,
        metrics=[
              faithfulness,
              answer_relevancy,
              answer_correctness#,
              #context_recall,
              #context_precision
              #context_relevancy
          ],
        llm=generator_llm,
        embeddings=ollama_emb,
        raise_exceptions=False
    )


## PROCESSO AUTOMATIZZATO CON PIU' LLM

In [ ]:
#SE GIA HO CALCOLATO IL TESTSET
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

In [ ]:
#ALTRIMENTI LO CARICO DAL CSV CHE HO PRESENTE E POI PROCEDO AL CALCOLO
import pandas as pd

# Carica il CSV in un nuovo DataFrame
loaded_df = pd.read_csv('test_dataChatGPT4O_FINALE.csv')

# Estrai le colonne 'question' e 'ground_truth'
questions = loaded_df["question"].to_list()
ground_truth = loaded_df["ground_truth"].to_list()

In [ ]:
from datasets import Dataset, Sequence, Value
from tqdm import tqdm
from langchain.llms import Ollama

# Lista dei modelli da processare
model_names = ["llama3.1", "qordmlwls/llama3.1-medical", "qwen2", "echelonify/med-qwen2", "mistral-nemo", "phi3", "gemma2"]

# Assicurati che questions e ground_truth siano definiti
# questions = testset.to_pandas()["question"].to_list()
# ground_truth = testset.to_pandas()["ground_truth"].to_list()

# Dizionario per memorizzare i risultati di ogni modello
resultsDict = {}

for model_name in model_names:
    print(f"Processing with model: {model_name}")
    # Inizializza l'LLM per il modello corrente
    llm = Ollama(model=model_name)
    
    # Inizializzazione del dizionario per il modello corrente
    data = {
        "question": [],
        "answer": [],
        "contexts": [],
        "ground_truth": ground_truth
    }
    
    # Utilizzo di tqdm per creare una barra di avanzamento
    for query in tqdm(questions, desc=f"Processing queries for {model_name}"):
        data["question"].append(query)
        data["answer"].append(rag_chain.invoke(query))
        data["contexts"].append([""])  # Contesto vuoto per ogni elemento
    
    # Convertire il dizionario in Dataset
    dataset = Dataset.from_dict(data)
    
    # Convertire in DataFrame e memorizzare i risultati
    resultsDict[f"dataset_df_{model_name}"] = dataset.to_pandas()

# Esempio di come accedere ai risultati:
# dataset_df_llama3_1 = resultsDict["dataset_df_llama3.1"]
# dataset_df_qwen2 = resultsDict["dataset_df_qwen2"]
# dataset_df_mistral_nemo = resultsDict["dataset_df_mistral-nemo"]

In [ ]:
import pickle
# Salva resultsDict su un file per poter riaccedere successivamente
with open('resultsDict_NORAG.pkl', 'wb') as f: #Cambianome per salvare più modelli 
    pickle.dump(resultsDict, f)

Per accedere al singolo dataset:

In [ ]:
resultsDict[f"dataset_df_phi3"]

AUTOMATIZZAZIONE DELLA VALUTAZIONE

In [ ]:
%%time
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from ragas import evaluate
from langchain.llms import Ollama
from datasets import Dataset

# Dizionario per memorizzare i risultati della valutazione
evaluation_results = {}

# Metriche da utilizzare per la valutazione
metrics = [
    faithfulness,
    answer_relevancy,
    answer_correctness

]

# Funzione per convertire DataFrame in Dataset
def convert_to_dataset(df):
    return Dataset.from_pandas(df)

# Itera su tutti i modelli presenti nel dizionario results
for model_name in ["llama3.1","qordmlwls/llama3.1-medical","qwen2","echelonify/med-qwen2","mistral-nemo","phi3","gemma2"]: #AGGIUNGERE ALTRI MODELLI...
    print(f"Evaluating model: {model_name}")


    # Valutazione per il dataset base
    resultBase = evaluate(
        dataset=convert_to_dataset(resultsDict[f"dataset_df_{model_name}"]),
        metrics=metrics,
        llm=azure_model_GPT4OMINI, #azure_model_GPT4OMINI o generator_llm  embeddings=azure_embeddings
        embeddings=azure_embeddings,
        raise_exceptions=False
    )

    # Memorizza i risultati della valutazione per questo modello
    evaluation_results[model_name] = {
        "base": resultBase
    }

# Stampa un riepilogo dei risultati
for model_name, model_results in evaluation_results.items():
    print(f"\nResults for {model_name}:")
    for dataset_type, result in model_results.items():
        print(f"  {dataset_type.capitalize()} Dataset:")
        for metric, score in result.items():
            print(f"    {metric}: {score:.4f}")

In [ ]:
import pickle
# Salva evaluation_results su un file per poter riaccedere successivamente
with open('evaluation_results_NORAG.pkl', 'wb') as f: #Cambianome per salvare più modelli 
    pickle.dump(evaluation_results, f)

In [ ]:
evaluation_results

## CARICO MODELLI PER LE ANALISI DA FILE PICKLE

### Carico il secondo pickle dove ho salvato i risultati degli altri modelli

In [ ]:
!pip install datasets==2.20.0

In [ ]:
import pickle
# Carica evaluation_results da un file
with open('evaluation_results_NORAG.pkl', 'rb') as f:
    evaluation_resultsNORAG = pickle.load(f)

In [ ]:
#LLAMA 3.1
LlamaNoRAG = evaluation_resultsNORAG["llama3.1"]["base"].to_pandas()

# LLAMA 3.1 MED
LlamaMedNoRAG = evaluation_resultsNORAG["qordmlwls/llama3.1-medical"]["base"].to_pandas()

# QWEN2
Qwen2NoRAG = evaluation_resultsNORAG["qwen2"]["base"].to_pandas()

#QWEN MED
QwenMedNoRAG = evaluation_resultsNORAG["echelonify/med-qwen2"]["base"].to_pandas()

#MISTRAL-NEMO
MistralNemoNoRAG = evaluation_resultsNORAG["mistral-nemo"]["base"].to_pandas()

#PHI3
Phi3NoRAG = evaluation_resultsNORAG["phi3"]["base"].to_pandas()

#GEMMA2
Gemma2NoRAG = evaluation_resultsNORAG["gemma2"]["base"].to_pandas()


In [ ]:
LlamaNoRAG

### Carico modello per analizzare i modelli basati su RAG

In [ ]:
import pickle
# Carica evaluation_results da un file
with open('evaluation_results_MODELLICOMP.pkl', 'rb') as f:
    evaluation_results = pickle.load(f)

In [ ]:
#LLAMA 3.1
LlamaBase = evaluation_results["llama3.1"]["base"].to_pandas()
LlamaMulti = evaluation_results["llama3.1"]["multi"].to_pandas()
LlamaEnsemble = evaluation_results["llama3.1"]["ensemble"].to_pandas()

# LLAMA 3.1 MED
LlamaMedBase = evaluation_results["qordmlwls/llama3.1-medical"]["base"].to_pandas()
LlamaMedMulti = evaluation_results["qordmlwls/llama3.1-medical"]["multi"].to_pandas()
LlamaMedEnsemble = evaluation_results["qordmlwls/llama3.1-medical"]["ensemble"].to_pandas()

# QWEN2
Qwen2Base = evaluation_results["qwen2"]["base"].to_pandas()
Qwen2Multi = evaluation_results["qwen2"]["multi"].to_pandas()
Qwen2Ensemble = evaluation_results["qwen2"]["ensemble"].to_pandas()

#QWEN MED
QwenMedBase = evaluation_results["echelonify/med-qwen2"]["base"].to_pandas()
QwenMedMulti = evaluation_results["echelonify/med-qwen2"]["multi"].to_pandas()
QwenMedEnsemble = evaluation_results["echelonify/med-qwen2"]["ensemble"].to_pandas()

#MISTRAL-NEMO
MistralNemoBase = evaluation_results["mistral-nemo"]["base"].to_pandas()
MistralNemoMulti = evaluation_results["mistral-nemo"]["multi"].to_pandas()
MistralNemoEnsemble = evaluation_results["mistral-nemo"]["ensemble"].to_pandas()

#PHI3
Phi3Base = evaluation_results["phi3"]["base"].to_pandas()
Phi3Multi = evaluation_results["phi3"]["multi"].to_pandas()
Phi3Ensemble = evaluation_results["phi3"]["ensemble"].to_pandas()

#GEMMA2
Gemma2Base = evaluation_results["gemma2"]["base"].to_pandas()
Gemma2Multi = evaluation_results["gemma2"]["multi"].to_pandas()
Gemma2Ensemble = evaluation_results["gemma2"]["ensemble"].to_pandas()

## ANALISI RISULTATI

CONTROLLO I VARI RISULTATI OTTENUTI PER SINGOLO RETRIEVER

In [ ]:
Phi3BaseSingolo = evaluation_results["phi3"]["base"].to_pandas()

In [ ]:
Phi3BaseSingolo

In [ ]:
evaluation_resultsNEW = evaluation_results

In [ ]:
evaluation_resultsNEW

In [ ]:
#resultMulti

In [ ]:
#resultEnsemble

PER CONTROLLARE SUL DIZIONARIO OTTENUTO DAL PROCESSO AUTOMATIZZATO ACCEDO PRIMA AL SINGOLO DATASET DEL LLM IN QUESTIONE PER VERIFICARE E POI ESEGUO ALTRI TEST

In [ ]:
# Carica ResultDict da un file
with open('resultsDict_4MODELLI.pkl', 'rb') as f:
    resultsDict = pickle.load(f)

In [ ]:
#SE VOGLIO TESTARE SU SINGOLO DATASET ALTRE METRICHE MI PRENDO IL DATASET DAL DIZIONARIO(SE NON E' CREATO, LO CARICO)- IN QUESTO CASO PER ESEMPIO E' PHI3 MA PER ALTRI BASTA MODIFICARE IL NOME
DataBasePhi3= convert_to_dataset(resultsDict["dataset_df_phi3"])
DataMultiPhi3= convert_to_dataset(resultsDict["advanced_retrieval_df_phi3"])
DataEnsemblePhi3= convert_to_dataset(resultsDict["ensemble_retrieval_df_phi3"])

### PER AGGIUNGERE NUOVA METRICA AL DIZIONARIO PRECEDENTE(MANUALE)

In [ ]:
new_metric_score = evaluate(DataBasePhi3, metrics=[harmfulness], llm=azure_model_GPT4OMINI, embeddings=azure_embeddings)
new_metric_df = new_metric_score.to_pandas()

In [ ]:
# Assumendo che la nuova metrica sia una singola colonna
new_metric_name = "harmfulness"  # Sostituisci con il nome effettivo della metrica
Phi3Base[new_metric_name] = new_metric_df[new_metric_name]

In [ ]:
Phi3Base[new_metric_name]

In [ ]:
evaluation_results["phi3"]["base"] = Dataset.from_pandas(Phi3Base)

In [ ]:
Phi3Base = evaluation_results["phi3"]["base"].to_pandas()

PER ELIMINARE:

In [ ]:
# Supponiamo che la colonna che vuoi eliminare si chiami "harmfulness"
column_to_remove = "harmfulness"

# Rimuovi la colonna da Phi3Base
Phi3Base = Phi3Base.drop(columns=[column_to_remove])

# Aggiorna il dizionario evaluation_results
evaluation_results["phi3"]["base"] = Dataset.from_pandas(Phi3Base)


In [ ]:
#USARE PER SOVRASCRIVERE IL FILE PICKLE PRECEDENTE AGGIORNANDOLO
with open('evaluation_results_4MODELLI_UPDATE.pkl', 'wb') as f:
    pickle.dump(evaluation_results, f)

In [ ]:
#PER TESTARE SINGOLA METRICA
#score = evaluate(DataBasePhi3,metrics=[harmfulness], llm=azure_model_GPT4OMINI, embeddings=azure_embeddings)
#score.to_pandas()

### AGGIUNTA DI METRICHE AUTOMATIZZATO PER TUTTI I MODELLI

In [ ]:
from tqdm import tqdm

def add_new_metric_to_all_models(resultsDict, evaluation_results, metric_function, metric_name, llm, embeddings):
    models = ["qordmlwls/llama3.1-medical", "qwen2", "mistral-nemo", "phi3"]
    approaches = {
        "base": "dataset_df_",
        "multi": "advanced_retrieval_df_",
        "ensemble": "ensemble_retrieval_df_"
    }
    
    total_iterations = len(models) * len(approaches)
    progress_bar = tqdm(total=total_iterations, desc="Processing models")

    for model in models:
        for approach, prefix in approaches.items():
            progress_bar.set_description(f"Processing {model} - {approach}")
            
            # Ottieni il dataset da resultsDict
            dataset_key = f"{prefix}{model}"
            current_dataset = convert_to_dataset(resultsDict[dataset_key])
            
            # Calcola la nuova metrica
            new_metric_score = evaluate(current_dataset, metrics=[metric_function], llm=llm, embeddings=embeddings)
            new_metric_df = new_metric_score.to_pandas()
            
            # Aggiunge la nuova metrica al DataFrame esistente in evaluation_results
            current_df = evaluation_results[model][approach].to_pandas()
            current_df[metric_name] = new_metric_df[metric_name]
            
            # Aggiorna il dizionario evaluation_results
            evaluation_results[model][approach] = Dataset.from_pandas(current_df)
            
            progress_bar.update(1)
    
    progress_bar.close()
    return evaluation_results

# Uso della funzione
new_evaluation_results = add_new_metric_to_all_models(
    resultsDict,
    evaluation_results, 
    harmfulness,  # La funzione metrica che hai definito
    "harmfulness",  # Il nome della nuova metrica
    azure_model_GPT4OMINI, 
    azure_embeddings
)

# Salva il dizionario aggiornato nel file pickle
with open('evaluation_results_4MODELLI_UPDATE.pkl', 'wb') as f:
    pickle.dump(new_evaluation_results, f)

In [ ]:
new_evaluation_results["qordmlwls/llama3.1-medical"]["base"]

In [ ]:
## CALCOLO MEDIA DELLE METRICHE

import numpy as np

def calculate_metrics_mean(dataset):
    metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness', 'context_recall', 'context_precision', 'harmfulness']
    
    means = {}
    for metric in metrics:
        values = dataset[metric]
        mean_value = np.mean(values)
        means[metric] = mean_value
    
    return means

# Uso della funzione
model = "qordmlwls/llama3.1-medical"
approach = "base"
dataset = new_evaluation_results[model][approach]

metric_means = calculate_metrics_mean(dataset)

# Stampa i risultati
for metric, mean in metric_means.items():
    print(f"{metric}: {mean:.4f}")

In [ ]:
# Carico per testare
with open('evaluation_results_4MODELLI_UPDATE.pkl', 'rb') as f:
    evaluation_update = pickle.load(f)

In [ ]:
evaluation_update

In [ ]:
# LLAMA 3.1 MED
LlamaMedBaseU = evaluation_update["qordmlwls/llama3.1-medical"]["base"].to_pandas()
LlamaMedMultiU = evaluation_update["qordmlwls/llama3.1-medical"]["multi"].to_pandas()
LlamaMedEnsembleU = evaluation_update["qordmlwls/llama3.1-medical"]["ensemble"].to_pandas()

# QWEN2
Qwen2BaseU = evaluation_update["qwen2"]["base"].to_pandas()
Qwen2MultiU = evaluation_update["qwen2"]["multi"].to_pandas()
Qwen2EnsembleU = evaluation_update["qwen2"]["ensemble"].to_pandas()

#MISTRAL-NEMO
MistralNemoBaseU = evaluation_update["mistral-nemo"]["base"].to_pandas()
MistralNemoMultiU = evaluation_update["mistral-nemo"]["multi"].to_pandas()
MistralNemoEnsembleU = evaluation_update["mistral-nemo"]["ensemble"].to_pandas()

#PHI3
Phi3BaseU = evaluation_update["phi3"]["base"].to_pandas()
Phi3MultiU = evaluation_update["phi3"]["multi"].to_pandas()
Phi3EnsembleU = evaluation_update["phi3"]["ensemble"].to_pandas()

## CREO IL DATASET DI CONFRONTO

In [ ]:
evaluation_results #RAG

In [ ]:
evaluation_resultsNORAG 

In [ ]:
import pandas as pd

# Funzione per creare un DataFrame di confronto per un singolo modello
def create_comparison_df(base_results, noRAG_results, model_name):
    # Creazione del DataFrame per BaseRetriever
    df_base = pd.DataFrame(list(base_results['base'].items()), columns=['Metric', 'BaseRetriever'])
    
    # Creazione del DataFrame per NoRAG retriever
    df_noRAG = pd.DataFrame(list(noRAG_results['base'].items()), columns=['Metric', 'NoRAGRetriever'])

    # Unire i due DataFrame basandosi sulla metrica
    df_merged = pd.merge(df_base, df_noRAG, on='Metric')

    # Aggiungere una colonna con il nome del modello
    df_merged['Model'] = model_name

    return df_merged

# Creare un DataFrame di confronto per ogni modello
comparison_dfs = {}
for model_name, base_results in evaluation_results.items():
    # Estrazione dei risultati corrispondenti dai dati NoRAG
    noRAG_results = evaluation_resultsNORAG[model_name]

    # Creazione del DataFrame di confronto
    comparison_dfs[model_name] = create_comparison_df(base_results, noRAG_results, model_name)

# Combinare tutti i DataFrame in un unico DataFrame di confronto
combined_df = pd.DataFrame()
for df in comparison_dfs.values():
    if combined_df.empty:
        combined_df = df
    else:
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Riorganizzare le colonne per avere 'Model' all'inizio, seguito dalle metriche
column_order = ['Model', 'Metric', 'BaseRetriever', 'NoRAGRetriever']
combined_df = combined_df[column_order]

# Visualizzare il DataFrame combinato
#print(combined_df)

# Opzionalmente, salvare il DataFrame in un file CSV
combined_df.to_csv('llm_rag_noRAG.csv', index=False)

# Visualizzazione del DataFrame
combined_df


## CREO LA MATRICE DI CONFUSIONE PER VALUTARE BENE IL MIO RAG

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

#df = result.to_pandas()

heatmap_data = QwenMedEnsemble[['faithfulness', 'answer_relevancy', 'answer_correctness']]

cmap = LinearSegmentedColormap.from_list('green_red', ['red', 'green'])

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", linewidths=.5, cmap=cmap)

plt.yticks(ticks=range(len(LlamaMedNoRAG['question'])), labels=LlamaMedNoRAG['question'], rotation=0)

plt.show()

### VALUTAZIONE VARI RETRIEVER CON GRAFICO A RADAR E BARRE PER SINGOLO MODELLO

In [ ]:
# MI ESTRAGGO I RISULTATI SINGOLARMENTTE
llamaBase = evaluation_results["llama3.1"]["base"].to_pandas()
llamaMulti = evaluation_results["llama3.1"]["multi"].to_pandas()
llamaEnsemble = evaluation_results["llama3.1"]["ensemble"].to_pandas()
#FARE PER ALTRI MODELLI...

In [ ]:
QwenMedBase = evaluation_results["echelonify/med-qwen2"]["base"].to_pandas()
QwenMedMulti = evaluation_results["echelonify/med-qwen2"]["multi"].to_pandas()
QwenMedEnsemble = evaluation_results["echelonify/med-qwen2"]["ensemble"].to_pandas()

In [ ]:
QwenMedBase[metrics] # VERIFICO LE METRICHE

In [ ]:
import plotly.graph_objects as go

metrics = ['answer_relevancy', 'faithfulness', 'answer_correctness']

fig = go.Figure()

for data, name, color in zip([Qwen2Base, Qwen2Multi, Qwen2Ensemble],
                             ['Qwen2 Base', 'Qwen2 Multi', 'Qwen2 Ensemble'],
                             ['blue', 'red', 'green']):
    fig.add_trace(go.Scatterpolar(
        r=data[metrics].values.flatten(),
        theta=metrics,
        fill='toself',
        name=name,
        line=dict(color=color, width=2),
        marker=dict(size=8, symbol='circle')
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            tickvals=[0.2, 0.4, 0.6, 0.8],
            ticktext=['20%', '40%', '60%', '80%']
        ),
        angularaxis=dict(
            tickfont=dict(size=12)
        )
    ),
    showlegend=True,
    legend=dict(font=dict(size=12)),
    title=dict(
        text="Retrieval Augmented Generation - Evaluation for Qwen2",
        font=dict(size=16)
    ),
    width=700,
    height=500
)

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

metrics = ['answer_relevancy', 'faithfulness', 'answer_correctness']
models = ['QwenMed Base', 'QwenMed Multi', 'QwenMed Ensemble']
colors = ['blue', 'red', 'green']

fig = make_subplots(rows=1, cols=3, subplot_titles=metrics)

for i, metric in enumerate(metrics, start=1):
    for j, (data, name, color) in enumerate(zip([QwenMedBase, QwenMedMulti, QwenMedEnsemble], models, colors)):
        fig.add_trace(
            go.Bar(
                x=[name], 
                y=[data[metric].values[0]], 
                name=name,
                marker_color=color,
                showlegend=i==1
            ),
            row=1, col=i
        )

fig.update_layout(
    title=dict(
        text="Retrieval Augmented Generation - Evaluation for QwenMed 2",
        font=dict(size=16)
    ),
    barmode='group',
    height=500,
    width=900,
    yaxis=dict(range=[0, 1], tickformat='.0%'),
    yaxis2=dict(range=[0, 1], tickformat='.0%'),
    yaxis3=dict(range=[0, 1], tickformat='.0%')
)

fig.show()

In [ ]:
import plotly.graph_objects as go

metrics = ['answer_relevancy', 'faithfulness', 'answer_correctness']
models = ['QwenMed Base', 'QwenMed Multi', 'QwenMed Ensemble']
colors = ['blue', 'red', 'green']

fig = go.Figure()

for data, name, color in zip([QwenMedBase, QwenMedMulti, QwenMedEnsemble], models, colors):
    fig.add_trace(go.Bar(
        name=name,
        x=metrics,
        y=data[metrics].values.flatten(),
        marker_color=color
    ))

fig.update_layout(
    title=dict(
        text="Retrieval Augmented Generation - Evaluation for QwenMed 2",
        font=dict(size=16)
    ),
    barmode='group',
    yaxis=dict(range=[0, 1], tickformat='.0%'),
    height=500,
    width=800
)

fig.show()

## GRAFICI

### GRAFICO DI COMPARAZIONE TRA DUE MODELLI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Funzione di analisi
def analysis(model1, model2):
    sns.set_style("whitegrid")
    fig, axs = plt.subplots(1, 3, figsize=(12, 5))
    columns = model1.columns
    for i, col in enumerate(columns):
        sns.kdeplot(data=model1[col].dropna(), legend=False, ax=axs[i], fill=True, label='Qwen2') #MODIFICA NOME MODELLO
        sns.kdeplot(data=model2[col].dropna(), legend=False, ax=axs[i], fill=True, label='Qwen2 Med') #MODIFICA NOME MODELLO
        axs[i].set_title(f'{col} scores distribution')
        axs[i].legend()
    plt.tight_layout()
    plt.show()


# Chiamata alla funzione di analisi
analysis(
    Qwen2Base[['faithfulness', 'answer_relevancy', 'answer_correctness']],
    QwenMedBase[['faithfulness', 'answer_relevancy', 'answer_correctness']]
)


### GRAFICO COMPARAZIONE PIU' MODELLI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def multi_model_analysis(*args):
    sns.set_style("whitegrid")
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']
    
    for i, metric in enumerate(metrics):
        for df, label in args:
            sns.kdeplot(data=df[metric].dropna(), ax=axs[i], fill=True, label=label)
        
        axs[i].set_title(f'{metric} scores distribution')
        axs[i].legend()
    
    plt.tight_layout()
    plt.show()

# Chiamata alla funzione di analisi
multi_model_analysis(
    (LlamaBase[metrics], 'Llama-Base'),
   # (LlamaMulti[metrics], 'Llama-Multi'),
   # (LlamaEnsemble[metrics], 'Llama-Ensemble'),
    (LlamaMedBase[metrics], 'LlamaMed-Base'),
   # (LlamaMedMulti[metrics], 'LlamaMed-Multi'),
   # (LlamaMedEnsemble[metrics], 'LlamaMed-Ensemble'),
    (Qwen2Base[metrics], 'Qwen2-Base'),
   # (Qwen2Multi[metrics], 'Qwen2-Multi'),
   # (Qwen2Ensemble[metrics], 'Qwen2-Ensemble'),
    (QwenMedBase[metrics], 'QwenMed-Base'),
   # (QwenMedMulti[metrics], 'QwenMed-Multi'),
   # (QwenMedEnsemble[metrics], 'QwenMed-Ensemble'),
    (MistralNemoBase[metrics], 'MistralNemo-Base'),
   #(MistralNemoMulti[metrics], 'MistralNemo-Multi'),
   # (MistralNemoEnsemble[metrics], 'MistralNemo-Ensemble'),
    (Phi3Base[metrics], 'Phi3-Base'),
   # (Phi3Multi[metrics], 'Phi3-Multi'),
   # (Phi3Ensemble[metrics], 'Phi3-Ensemble')
    (Gemma2Base[metrics], 'Gemma2-Base')
   # (Gemma2Multi[metrics], 'Gemma2-Multi'),
   # (Gemma2Ensemble[metrics], 'Gemma2-Ensemble')
)

## ALTRI GRAFICI DA PROVATI

### HEATMAP

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Preparazione dei dati
models = ['Llama','LlamaMed', 'Qwen2','QwenMed', 'MistralNemo', 'Phi3','Gemma2'] #Modelli da testare : 'Qwen2','QwenMed', 'MistralNemo', 'Phi3','Gemma2'
retrievers = ['Base','NoRAG' ]#'Multi', 'Ensemble'
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']

data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([f"{model}-{retriever}"] + list(avg_scores))

df = pd.DataFrame(data, columns=['Model'] + metrics)
df_pivot = df.set_index('Model').T

# Creazione della heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_pivot, annot=True, cmap='YlGnBu', fmt='.3f')
plt.title('Model Comparison - Average Scores')
plt.tight_layout()
plt.show()

### RADAR CHART

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def radar_chart(data, categories, title):
    N = len(categories)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]

    ax = plt.subplot(111, polar=True)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], categories)

    for i, (model, scores) in enumerate(data.items()):
        values = scores + scores[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=model)
        ax.fill(angles, values, alpha=0.1)

    plt.title(title)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

# Preparazione dei dati
models = ['LlamaMed']#'Qwen2','QwenMed', 'MistralNemo', 'Phi3'
retrievers = ['Base']#'Multi', 'Ensemble
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']

data = {}
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data[f"{model}-{retriever}"] = list(avg_scores)

# Creazione del radar chart
plt.figure(figsize=(10, 10))
radar_chart(data, metrics, 'Model Comparison - Average Scores')
plt.tight_layout()
plt.show()

### BAR PLOT

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Preparazione dei dati
models = ['Llama','LlamaMed']#'Qwen2','QwenMed', 'MistralNemo', 'Phi3','Gemma2'
retrievers = ['Base', 'Multi', 'Ensemble']
metrics = ['faithfulness', 'answer_relevancy','answer_correctness' ]#'answer_relevancy''faithfulness',

data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([f"{model}-{retriever}"] + list(avg_scores))

df = pd.DataFrame(data, columns=['Model'] + metrics)

# Creazione del bar plot
df.set_index('Model').plot(kind='bar', figsize=(15, 8))
plt.title('Model Comparison - Average Scores')
plt.xlabel('Model-Retriever')
plt.ylabel('Average Score')
plt.legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### CON ETICHETTE

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Preparazione dei dati
models = ['Llama', 'LlamaMed']  # 'Qwen2','QwenMed', 'MistralNemo', 'Phi3','Gemma2'
retrievers = ['Base', 'Multi', 'Ensemble']
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']
data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([f"{model}-{retriever}"] + list(avg_scores))
df = pd.DataFrame(data, columns=['Model'] + metrics)

# Creazione del bar plot
ax = df.set_index('Model').plot(kind='bar', figsize=(15, 8))
plt.title('Model Comparison - Average Scores')
plt.xlabel('Model-Retriever')
plt.ylabel('Average Score')
plt.legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

# Aggiunta delle etichette sopra ogni barra
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f', padding=3)

# Aggiustamento del layout per evitare sovrapposizioni
plt.tight_layout()
plt.show()

### RAG_BASERETRIEVER VS NORAG

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Preparazione dei dati
models = ['Llama', 'LlamaMed', 'Qwen2', 'QwenMed', 'MistralNemo', 'Phi3', 'Gemma2']
retrievers = ['Base', 'NoRAG']
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']

data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([model, retriever] + list(avg_scores))

df = pd.DataFrame(data, columns=['Model', 'Retriever'] + metrics)

# Creazione del grafico a barre raggruppate
plt.figure(figsize=(14, 8))
x = np.arange(len(models))
width = 0.35

base_scores = df[df['Retriever'] == 'Base']['answer_correctness'].values
norag_scores = df[df['Retriever'] == 'NoRAG']['answer_correctness'].values

plt.bar(x - width/2, base_scores, width, label='RAG', color='skyblue')
plt.bar(x + width/2, norag_scores, width, label='NoRAG', color='lightgreen')

plt.xlabel('Models')
plt.ylabel('Answer Correctness Score')
plt.title('Comparison of RAG vs NoRAG - Answer Correctness')
plt.xticks(x, models, rotation=45)
plt.legend()

# Aggiungi i valori sopra le barre
for i, v in enumerate(base_scores):
    plt.text(i - width/2, v, f'{v:.3f}', ha='center', va='bottom')
for i, v in enumerate(norag_scores):
    plt.text(i + width/2, v, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

Creo i tre grafici

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

# Creazione della cartella immaginiPDF se non esiste
if not os.path.exists('immaginiPDF'):
    os.makedirs('immaginiPDF')

# Preparazione dei dati
models = ['Llama', 'LlamaMed', 'Qwen2', 'QwenMed', 'MistralNemo', 'Phi3', 'Gemma2']
retrievers = ['Base', 'NoRAG']
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']
data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([model, retriever] + list(avg_scores))
df = pd.DataFrame(data, columns=['Model', 'Retriever'] + metrics)

# Funzione per creare il grafico
def create_plot(metric):
    plt.figure(figsize=(10, 6))
    plt.rcParams['font.size'] = 10
    x = np.arange(len(models))
    width = 0.35
    base_scores = df[df['Retriever'] == 'Base'][metric].values
    norag_scores = df[df['Retriever'] == 'NoRAG'][metric].values
    plt.bar(x - width/2, base_scores, width, label='RAG', color='skyblue')
    plt.bar(x + width/2, norag_scores, width, label='NoRAG', color='lightgreen')
    plt.xlabel('Models')
    plt.ylabel(f'{metric.capitalize()} Score')
    plt.title(f'Comparison of RAG vs NoRAG - {metric.capitalize()}')
    plt.xticks(x, models, rotation=45)
    plt.legend()
    # Aggiungi i valori sopra le barre
    for i, v in enumerate(base_scores):
        plt.text(i - width/2, v, f'{v:.3f}', ha='center', va='bottom')
    for i, v in enumerate(norag_scores):
        plt.text(i + width/2, v, f'{v:.3f}', ha='center', va='bottom')
    plt.tight_layout()
    
    # Salva il grafico in PDF
    pdf_path = os.path.join('immaginiPDF', f'rag_vs_norag_{metric}.pdf')
    plt.savefig(pdf_path, format='pdf', dpi=300, bbox_inches='tight')
    print(f"Grafico salvato come {pdf_path}")
    
    # Visualizza il grafico
    plt.show()

# Crea un grafico per ciascuna metrica
for metric in metrics:
    create_plot(metric)

print("I grafici sono stati salvati nella cartella 'immaginiPDF' come 'rag_vs_norag_faithfulness.pdf', 'rag_vs_norag_answer_relevancy.pdf', e 'rag_vs_norag_answer_correctness.pdf'.")

### RAG_MULTIRETRIEVER VS NORAG

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Preparazione dei dati
models = ['Llama', 'LlamaMed', 'Qwen2', 'QwenMed', 'MistralNemo', 'Phi3', 'Gemma2']
retrievers = ['Multi', 'NoRAG']
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']

data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([model, retriever] + list(avg_scores))

df = pd.DataFrame(data, columns=['Model', 'Retriever'] + metrics)

# Funzione per creare il grafico
def create_plot(metric):
    plt.figure(figsize=(10, 6))
    plt.rcParams['font.size'] = 10
    x = np.arange(len(models))
    width = 0.35

    multi_scores = df[df['Retriever'] == 'Multi'][metric].values
    norag_scores = df[df['Retriever'] == 'NoRAG'][metric].values

    plt.bar(x - width/2, multi_scores, width, label='RAG Multi', color='skyblue')
    plt.bar(x + width/2, norag_scores, width, label='NoRAG', color='lightgreen')

    plt.xlabel('Models')
    plt.ylabel(f'{metric.capitalize()} Score')
    plt.title(f'Comparison of RAG vs NoRAG - {metric.capitalize()}')
    plt.xticks(x, models, rotation=45)
    plt.legend()
    # Aggiungere spazio per la legenda
    plt.subplots_adjust(right=0.75)
    # Posizionare la legenda a destra fuori dal grafico
    plt.legend(loc='center left', bbox_to_anchor=(1, 1))
    # Aggiungi i valori sopra le barre
    for i, v in enumerate(multi_scores):
        plt.text(i - width/2, v, f'{v:.3f}', ha='center', va='bottom')
    for i, v in enumerate(norag_scores):
        plt.text(i + width/2, v, f'{v:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# Crea un grafico per ciascuna metrica
for metric in metrics:
    create_plot(metric)

### RAG_ENSEMBLERETRIEVER VS NORAG

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Preparazione dei dati
models = ['Llama', 'LlamaMed', 'Qwen2', 'QwenMed', 'MistralNemo', 'Phi3', 'Gemma2']
retrievers = ['Ensemble', 'NoRAG']
metrics = ['faithfulness', 'answer_relevancy', 'answer_correctness']

data = []
for model in models:
    for retriever in retrievers:
        model_data = globals()[f"{model}{retriever}"]
        avg_scores = model_data[metrics].mean()
        data.append([model, retriever] + list(avg_scores))

df = pd.DataFrame(data, columns=['Model', 'Retriever'] + metrics)

# Funzione per creare il grafico
def create_plot(metric):
    plt.figure(figsize=(10, 6))
    plt.rcParams['font.size'] = 10
    x = np.arange(len(models))
    width = 0.35

    ensemble_scores = df[df['Retriever'] == 'Ensemble'][metric].values
    norag_scores = df[df['Retriever'] == 'NoRAG'][metric].values

    plt.bar(x - width/2, ensemble_scores, width, label='RAG Ensemble', color='skyblue')
    plt.bar(x + width/2, norag_scores, width, label='NoRAG', color='lightgreen')

    plt.xlabel('Models')
    plt.ylabel(f'{metric.capitalize()} Score')
    plt.title(f'Comparison of RAG vs NoRAG - {metric.capitalize()}')
    plt.xticks(x, models, rotation=45)
    plt.legend()
    # Aggiungere spazio per la legenda
    plt.subplots_adjust(right=0.75)
    # Posizionare la legenda a destra fuori dal grafico
    plt.legend(loc='center left', bbox_to_anchor=(1, 1))
    # Aggiungi i valori sopra le barre
    for i, v in enumerate(ensemble_scores):
        plt.text(i - width/2, v, f'{v:.3f}', ha='center', va='bottom')
    for i, v in enumerate(norag_scores):
        plt.text(i + width/2, v, f'{v:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# Crea un grafico per ciascuna metrica
for metric in metrics:
    create_plot(metric)